工业品出厂价格指数ppi预测股票市场收益率（选取2012年-2022年科技板块股票数据）

准备工作

In [9]:
import numpy as np # 数据处理最重要的模块
import pandas as pd # 数据处理最重要的模块
import matplotlib.pyplot as plt  # 画图模块
import scipy.stats as stats # 统计模块
import scipy
from datetime import datetime # 时间模块
from IPython.core.interactiveshell import InteractiveShell # jupyter运行输出的模块
import statsmodels.formula.api as smf  # OLS regression

#输出矢量图 渲染矢量图 是一个魔法函数（Magic Functions）内嵌绘图
%matplotlib inline 
%config InlineBackend.figure_format = 'svg'

#显示每一个运行结果
InteractiveShell.ast_node_interactivity = 'all'

#设置行不限制数量
#pd.set_option('display.max_rows',None)

#设置列不限制数量
pd.set_option('display.max_columns', None)


导入数据

In [8]:
data = pd.read_csv('数据\科技板块股票.csv')
data['Day'] = pd.to_datetime(data['Day'],format='%Y/%m/%d')
data.set_index('Day', inplace = True)
data.sort_values(by = ['Day'],axis=0, ascending=True)
data

NameError: name 'pd' is not defined

In [10]:
ppi=pd.read_csv('数据\工业品出厂价格指数ppi.csv')#读取工业品出厂价格指数ppi数据
ppi['month']=pd.to_datetime(ppi['month'],format = '%Y/%m/%d')#将数据中的时间转化为日期格式
ppi.set_index('month',inplace= True) #设置索引
ppi.sort_values(by=['month'],axis=0,ascending=True)#选出其中的月份数据

NameError: name 'pd' is not defined

计算收益率股票的收益率

股票的收益率的计算一般使用的是 **收盘价** 来计算。


* Raw Return: $R_t = \frac{p_t - p_{t-1}}{p_{t-1}} $
* Log Return: $r_t = log(p_t) - log(p_{t-1})$

自然对数收益率与原始收益率的关系：

$r_t = log(1 + R_t)$

* 在实际数据中，我们常用收盘价来计算收益率。为什么？
* 对数收益率和原始收益率的区别在哪里？分别用在什么样的场景呢？

## 复利的计算
设某资产的的初始值为 $C$, 名义上的年利率为 $r$ ， 但是在一年内分成 $m$ 次付息，理论上每次付息 $C \frac{r}{m}$, 最终的资产净值应为 

$C+C \frac{r}{m} \times m=C(1+r)$; 

但是，因为提前付息，所以提前支付的利息也进入账户增值，从第二次付息开始，支付的利息就超过了 $C \frac{r}{m}$, 使得一年后的净值要高于 $C(1+r)$。 一年后的净值为

$C\left(1+\frac{r}{m}\right)^{m}$

当 $m \rightarrow \infty$ 时，由极限 $\lim _{x \rightarrow+\infty}\left(1+\frac{1}{x}\right)^{x}=e$, 可知

$\lim _{m \rightarrow \infty} C\left(1+\frac{r}{m}\right)^{m}=\lim _{m \rightarrow \infty} C\left[\left(1+\frac{r}{m}\right)^{\frac{m}{r}}\right]^{r}=C e^{r}$

这时 $r$ 称为连续复利， 它也对应某个时间单位（一般是年)， $R=e^{r}-1$ 是连续复利 $r$ 对应的实际利率, $r$ 与 $R$ 的关系为

$R=e^{r}-1, \quad r=\ln (1+R)$

In [ ]:
index = data['2012-12':'2022-12'].copy()
index['Close'] = pd.to_numeric(index['Close'])
index['Preclose'] = pd.to_numeric(index['Preclose'])
index

NameError: name 'data' is not defined

In [11]:
# 计算科技板块股票日收益率 两种：
index['Raw_return'] = index['Close'] / index['Preclose'] - 1
index['Log_return'] = np.log(index['Close']) - np.log(index['Preclose'])
index['Stupid_return'] = 0
index

NameError: name 'index' is not defined

In [ ]:
for i in range(0,):
    index.iloc[i,9] = index.iloc[i,4] / index.iloc[i,0] - 1

index

In [ ]:
# 两种计算方式的差
index['delta_return'] = index['Raw_return'] - index['Log_return']
print('平均来看，两种方式计算的收益率之间的区别为:',np.mean(index['delta_return'])*100,"%")

# 计算月、季和年度收益率 Calculate Monthly Quarterly and Yearly Return


## 月度收益率
$t$ 月的收益率使用该月月末的收盘价 和 上个月$t - 1$月末的收盘价来计算

* Raw Return: $R_t = \frac{p_t - p_{t-1}}{p_{t-1}} $
* Log Return: $r_t = log(p_t) - log(p_{t-1})$

### 使用resample函数

#### 对数回报率求和

In [ ]:
Month_data = index.resample('m')['Log_return'].sum().to_frame() # R语言非常常见
Month_data['Raw_Return'] = np.exp(Month_data['Log_return']) - 1
Month_data

#### 取月末的价格计算

In [ ]:
Month_data = index.resample('m')['Close'].last().to_frame()
Month_data['Preclose'] = Month_data['Close'].shift(1)
Month_data['Log_return'] = np.log(Month_data['Close']) - np.log(Month_data['Preclose'])
Month_data

### 使用groupby函数

In [ ]:
# “2012-12-31”日期格式 里面的year年份 month月份 day 直接提出取来
index2 = index
index2['year'] = index2.index.year
index2['month'] = index2.index.month
index2
# 使用的时间、日期格式提取 字符串提出的方式 前四个字符当作年份 6-7字符是月份 提取出来的是字符串 变成数值

In [ ]:
Month_data2 = pd.DataFrame(index.groupby(['year','month'])['Log_return'].sum())
Month_data2

In [ ]:
Month_data2 = pd.DataFrame(
    index.groupby(['year', 'month'])['Log_return'].apply(lambda x: sum(x)))
    # sum函数是python内置函数
Month_data2

# 描述性统计 Statistical Summary

In [ ]:
daily_data = data['2012-12':'2022-12'].copy()
daily_data['Close'] = pd.to_numeric(daily_data['Close'])
daily_data['Preclose'] = pd.to_numeric(daily_data['Preclose'])
# 计算科技板块股票收益率 两种：
daily_data['Raw_return'] = daily_data['Close'] / daily_data['Preclose'] - 1
daily_data['Log_return'] = np.log(daily_data['Close']) - np.log(daily_data['Preclose'])
daily_data

In [ ]:
Month_data = daily_data.resample('m')['Log_return'].sum().to_frame()
Month_data['Raw_return'] = np.exp(Month_data['Log_return']) - 1
Month_data

In [ ]:
Quarter_data = daily_data.resample('Q')['Log_return'].sum().to_frame()
Quarter_data['Raw_return'] = np.exp(Quarter_data['Log_return']) - 1
Quarter_data

In [ ]:
Year_data = daily_data.resample('Y')['Log_return'].sum().to_frame()
Year_data['Raw_return'] = np.exp(Year_data['Log_return']) - 1
Year_data

# 分位数 quantile

分位数（Quantile），亦称分位点，是指将一个随机变量的概率分布范围分为几个等份的数值点，常用的有中位数（即二分位数）、四分位数、百分位数等。

举例说明，我们班上有10名同学，他们的期末成绩是

【20，24，38，34，43，56，76，86，98，100】

如果期末考试要让40%的不通过，60%的人通过，分数线是多少呢？

常见的分类方法：

- 中位数 Median
  - 当N是奇数时
    - $$m_{0.5}=X_{(N+1) / 2}$$
  - 当N是偶数时
    - $$m_{0.5}=\frac{X_{(N / 2)}+X_{(N / 2+1)}}{2}$$
- 四分位数 Quartile
- 10分位数 常用在股票投资策略中

Some q-quantiles have special names:

- The only 2-quantile is called the median
- The 3-quantiles are called tertiles or terciles 
- The 4-quantiles are called quartiles 
- The 5-quantiles are called quintiles 
- The 6-quantiles are called sextiles 
- The 7-quantiles are called septiles
- The 8-quantiles are called octiles
- The 10-quantiles are called deciles
- The 12-quantiles are called duo-deciles or dodeciles
- The 16-quantiles are called hexadeciles
- The 20-quantiles are called ventiles, vigintiles, or demi-deciles 
- The 100-quantiles are called percentiles

In [ ]:
from statistics import quantiles

# quantiles函数里面的参数需要注意
quantiles(daily_data['2012-12':'2022-12']['Raw_return'], n = 10, method='inclusive')

In [ ]:
quantiles(Month_data['2012-12':'2022-12']['Raw_return'], n = 4)

In [ ]:
quantiles(daily_data['2012-12':'2022-12']['Raw_return'], n = 10, method='exclusive')

In [ ]:
seq = np.linspace(0,1,11)
seq

In [ ]:
np.quantile(daily_data['2012-12':'2022-12']['Raw_return'],q=seq)

In [ ]:
arr = np.array([1,2,3,4,5,6,7,8,9,10,11,12]) # 共12个数
np.quantile(arr, (0.25, 0.50, 0.75))

In [ ]:
quantiles(arr, n = 4, method='exclusive')

In [ ]:
quantiles(arr, n = 4, method='inclusive')

# 波动率 Variance

$$\sigma^2=\frac{\sum(X-\mu)^2}{N}$$

In [ ]:
np.var(daily_data['2012-12':'2022-12']['Raw_return'])

In [ ]:
m = np.mean(daily_data['2012-12':'2022-12']['Raw_return'])
sum((daily_data['2012-12':'2022-12']['Raw_return'] - m)**2) / len(daily_data['2012-12':'2022-12']['Raw_return'])

In [ ]:
np.std(daily_data['2012-12':'2022-12']['Raw_return'])

In [ ]:
from math import sqrt

sqrt(np.var(daily_data['2012-12':'2022-12']['Raw_return']))

每个月的波动率

In [ ]:
monthly_var = daily_data.resample('m')['Raw_return'].var().to_frame()
monthly_var.rename(columns={'Raw_return':'Variance'},inplace=True)
monthly_var

# 正态分布

若随机变量X服从一个数学期望为 $\mu$ 、方差为 $\sigma^{2}$ 的正态分布，记为 $\mathrm{N}\left(\mu, \sigma^{2}\right)$ 。其概率密度函数为正态分布的期望值决定了 其位置，其标准差$\sigma$决定了分布的幅度。当 $\mu=0, \sigma=1$ 时的正态分布是标准正态分布。

若随机变量 $X$ 服从一个位置参数为 $\mu$ 、尺度参数为 $\sigma$ 的概率分布，且其概率密度函数为 
$$
f(x)=\frac{1}{\sqrt{2 \pi} \sigma} \exp \left(-\frac{(x-\mu)^{2}}{2 \sigma^{2}}\right)
$$
则这个随机变量就称为正态随机变量, 正态随机变量服从的分布就称为正态分布, 记作 $X \sim N\left(\mu, \sigma^{2}\right)$, 读作 $X$ 服从 $N\left(\mu, \sigma^{2}\right)$, 或 $X$ 服从正态分布。

 若 $X \sim N\left(\mu, \sigma^{2}\right), Y=\frac{X-\mu}{\sigma} \sim N(0,1)$.
 
 当 $\mu=0, \sigma=1$ 时，正态分布就成为标准正态分布
$$
f(x)=\frac{1}{\sqrt{2 \pi}} e^{\left(-\frac{x^{2}}{2}\right)}
$$

# 偏度和峰度

* 偏度（Skewness）是统计数据分布偏斜方向和程度的度量，是统计数据分布非对称程度的数字特征。定义上偏度是样本的三阶标准化矩。
$$
S=\frac{1}{n} \sum_{i=1}^{n}\left[\left(\frac{X_{i}-\mu}{\sigma}\right)^{3}\right]
$$

其中 $\mu$ 是均值， $\sigma$ 是标准差。定义中包括正态分布（偏度=0），右偏分布（正偏，>0）,左偏分布（负偏，<0）


* 峰度（Kurtosis）表征概率密度分布曲线在平均值处峰度高低的特征。直观来看，峰度反应了峰部的尖度。其计算方法为
   $$K=\frac{1}{n} \sum_{i=1}^{n}\left[\left(\frac{X_{i}-\mu}{\sigma}\right)^{4}\right]$$
    其中 $\mu$ 是均值， $\sigma$ 是标准差。定义中包括正态分布（峰度=3），厚尾（峰度>3）,瘦尾（峰度<3）。

In [ ]:
# Daily data
daily_data['2012-12':'2022-12']['Raw_return'].skew()
daily_data['2012-12':'2022-12']['Raw_return'].kurt()
scipy.stats.kurtosis(daily_data['2012-12':'2022-12']['Raw_return'],fisher=False) - 3

In [ ]:
# Monthly data
Month_data['2012-12':'2022-12']['Raw_return'].skew()
Month_data['2012-12':'2022-12']['Raw_return'].kurt()

In [ ]:
# Quarterly data
Quarter_data['2012-12':'2022-12']['Raw_return'].skew()
Quarter_data['2012-12':'2022-12']['Raw_return'].kurt()

In [ ]:
Year_data['2012-12':'2022-12']['Raw_return'].skew()
Year_data['2012-12':'2022-12']['Raw_return'].kurt()

# 标准正态分布的直方图

In [ ]:
# 根据均值、标准差,求指定范围的正态分布概率值

def normfun(x, mu, sigma):
  pdf = np.exp(-((x - mu)**2)/(2*sigma**2)) / (sigma * np.sqrt(2*np.pi))
  return pdf


# 生成高斯分布的概率密度随机数
result = np.random.normal(0, 1, 1000000) # mean 0 and standard deviation 1

# np.arange()
# 函数返回一个有终点和起点的固定步长的排列，如[1,2,3,4,5]，起点是1，终点是6，步长为1。
# 参数个数情况： np.arange()函数分为一个参数，两个参数，三个参数三种情况
# 1）一个参数时，参数值为终点，起点取默认值0，步长取默认值1。
# 2）两个参数时，第一个参数为起点，第二个参数为终点，步长取默认值1。
# 3）三个参数时，第一个参数为起点，第二个参数为终点，第三个参数为步长。其中步长支持小数

x = np.arange(min(result), max(result),0.01)

# 设定 y 轴，载入刚才的正态分布函数
y = normfun(x, result.mean(), result.std())
plt.plot(x, y) # 这里画出理论的正态分布概率曲线
 
# 这里画出实际的参数概率与取值关系
fig = plt.hist(result, bins=100, rwidth=1, density=True) # bins个柱状图,宽度是rwidth(0~1),=1没有缝隙
plt.title('distribution')
plt.xlabel('temperature')
plt.ylabel('probability')
# 输出
plt.show() # 最后图片的概率和不为1是因为正态分布是从负无穷到正无穷,这里指截取了数据最小值到最大值的分布

# 在日收益率直方图上添加正态分布密度曲线

In [ ]:
retmean = daily_data['2012-12':'2022-12']['Raw_return'].mean()
retstd = daily_data['2012-12':'2022-12']['Raw_return'].std()
retmin = daily_data['2012-12':'2022-12']['Raw_return'].min()
retmax = daily_data['2012-12':'2022-12']['Raw_return'].max()


def normfun(x, mu, sigma):
    pdf = np.exp(-((x - mu)**2) /
                 (2 * sigma**2)) / (sigma * np.sqrt(2 * np.pi))
    return pdf


x = np.arange(retmin, retmax, 0.001) # 0.001 0.002 0.003 0.004.。。。 0.099 0.100  0.101
y = normfun(x, retmean, retstd)
fig = plt.figure(figsize=(10, 5))
plt.plot(x, y)

fig = plt.hist(daily_data['2012-12':'2022-12']['Raw_return'], # index daily market excess return
               bins=1000,
               histtype='bar',
               color='r',
               alpha=0.9)
plt.xticks(rotation=90, fontsize=8) # x轴显示方法
plt.title('Distribution')
plt.xlabel('Market Return From 2012 to 2022')
plt.ylabel('Probability')

# 在月收益率直方图上添加正态分布密

In [ ]:
retmean = Month_data['2012-12':'2022-12']['Raw_return'].mean()
retstd = Month_data['2012-12':'2022-12']['Raw_return'].std()
retmin = Month_data['2012-12':'2022-12']['Raw_return'].min()
retmax = Month_data['2012-12':'2022-12']['Raw_return'].max()


def normfun(x, mu, sigma):
    pdf = np.exp(-((x - mu)**2) /
                 (2 * sigma**2)) / (sigma * np.sqrt(2 * np.pi))
    return pdf


x = np.arange(retmin, retmax, 0.001)
y = normfun(x, retmean, retstd)
fig = plt.figure(figsize=(10, 5)) # 图片大小 10，5
plt.plot(x, y)

fig = plt.hist(Month_data['2012-12':'2022-12']['Raw_return'],
               bins=100,
               histtype='bar',
               color='r',
               alpha=0.9)
plt.xticks(rotation=90, fontsize=8)
plt.title('Distribution')
plt.xlabel('Market Return From 2012 to 2022')
plt.ylabel('Probability')

# 平稳性 stationary

In [ ]:
from statsmodels.tsa.stattools import adfuller as ADF
print('原始序列的ADF检验结果为:', ADF(daily_data['2012-12':'2022-12']['Raw_return']))

## 固定比例法

这种方法非常容易理解，我们把上下1%的值重新设置,若大于99%分位数的数值，则将其设置为99%分位数值，若低于1%分位数的数值，则将其重新设置为1%分位数值。

In [ ]:
Month_data['2012-12':'2022-12']['Raw_return'].max()
Month_data['2012-12':'2022-12']['Raw_return'].min()

In [ ]:
Month_data_fix = Month_data['2012-12':'2022-12'].copy()
Month_data_fix

In [ ]:
Month_data_fix[Month_data_fix['Raw_return'] > Month_data_fix['Raw_return'].quantile(
    0.99)] = Month_data_fix['Raw_return'].quantile(0.99)
Month_data_fix[Month_data_fix['Raw_return'] < Month_data_fix['Raw_return'].quantile(
    0.01)] = Month_data_fix['Raw_return'].quantile(0.01)
Month_data_fix

In [ ]:
Month_data['2012-12':'2020-12']['Raw_return'].describe().round(8)
Month_data_fix['2012-12':'2020-12']['Raw_return'].describe().round(8)

## 均值标准差法

这种想法的思路来自正态分布,假设$X \sim N\left(\mu, \sigma^{2}\right)$，那么


$P(|X-\mu|>k * \sigma)= \begin{cases}0.317, & k=1 \\ 0.046, & k=2 \\ 0.003, & k=3\end{cases}$

通常把**3倍**标准差之外的值都视为异常值，不过要注意的是样本均值和样本标准差都不是稳健统计量，其计算本身受极值的影响就非常大，所以可能会出现一种情况，那就是我们从数据分布图上能非常明显地看到异常点，但按照上面的计算方法，这个异常点可能仍在均值3倍标准差的范围内。因此按照这种方法剔除掉异常值后,需要重新观察数据的分布情况，看是否仍然存在显著异常点,若存在则继续重复上述步骤寻找异常点.

In [ ]:
Month_data_ms = Month_data['2012-12':'2022-12'].copy()

Month_data_ms[Month_data_ms['Raw_return'] >= Month_data_ms['Raw_return'].mean() +
         3 * Month_data_ms['Raw_return'].std()] = Month_data_ms['Raw_return'].mean(
         ) + 3 * Month_data_ms['Raw_return'].std()

Month_data_ms[Month_data_ms['Raw_return'] <= Month_data_ms['Raw_return'].mean() -
         3 * Month_data_ms['Raw_return'].std()] = Month_data_ms['Raw_return'].mean(
         ) - 3 * Month_data_ms['Raw_return'].std()
Month_data_ms

In [ ]:
Month_data['2012-12':'2022-12']['Raw_return'].describe().round(8)
Month_data_ms['2012-12':'2022-12']['Raw_return'].describe().round(8)

## MAD法

**MAD法**是针对均值标准差方法的改进,把均值和标准差替换成稳健统计量，样本均值用样本中位数代替，样本标准差用样本MAD( median absolute deviation)代替：

\begin{aligned}
&\operatorname{md}=\operatorname{median}\left(x_{i}, i=1,2, \cdots, n\right) \\
&\mathrm{MAD}=\operatorname{mean}\left(\left|x_{i}-\mathrm{md}\right|, i=1,2, \cdots, n\right)
\end{aligned}

一般将偏离中位数3倍以上的数据作为异常值，和均值标淮差法相比，其中位数和MAD不受异常值的影响。

In [ ]:
Month_data_median = Month_data['2012-12':'2022-12'].copy()

median = np.median(Month_data_median['Raw_return'])
MAD = np.mean(abs(Month_data_median['Raw_return'] - median))

Month_data_median[Month_data_median['Raw_return'] >= 3 * MAD + median] = 3 * MAD + median
Month_data_median[Month_data_median['Raw_return'] <= -3 * MAD + median] = -3 * MAD + median

Month_data_median

In [ ]:
Month_data['2012-12':'2022-12']['Raw_return'].describe().round(8)
Month_data_median['2012-12':'2022-12']['Raw_return'].describe().round(8)

# 参数假设检验

In [ ]:
Month_data['2012-12':'2022-12']['Raw_return'].mean()

## 基本理论

为了推断总体的某些性质，我们会提出;总体性质的各种假设。假设检验就是根据样本提供的信息对所提出的假设作出判断的过程。

原假设是我们有怀疑，想要拒绝的假设，记为$H_0$,备择假设是我们拒绝了原假设后得到的结论,记为$H_a$.

假设都是关于总体参数的，例如，我们想知道总体均值是否等于某个常数$\mu_0$，那么原假设是：$H_0: \mu = \mu_0$，则备择假设是：$H_a: \mu \ne \mu_0$.

上面这种假设，我们称为双尾检验，因为备择假设是双边的。

下面两种假设检验称为单尾检验：

\begin{array}{ll}
H_{0}: \mu \geqslant \mu_{0} & H_{\alpha}: \mu<\mu_{0} \\
H_{0}: \mu \leqslant \mu_{0} & H_{\alpha}: \mu>\mu_{0}
\end{array}

注意：无论是单尾还是双尾检验，等号永远都在原假设一边，这是用来判断原来假设的唯一标准.

In [ ]:
daily_data['2012-12':'2022-12']['Raw_return'].mean()

In [ ]:
stats.ttest_1samp(daily_data['2012-12':'2022-12']['Raw_return'],0)

In [ ]:
stats.ttest_1samp(Month_data['2012-12':'2022-12']['Raw_return'],0)

# 一元线性回归分析的基本理论

## 简单的一元OLS回归


动量效应（Momentum effect）一般又称“惯性效应”。动量效应是由Jegadeesh和Titman（1993）提出的，是指股票的收益率有延续原来的运动方向的趋势，即过去一段时间收益率较高的股票在未来获得的收益率仍会高于过去收益率较低的股票。

在下面的模型里面，我们考虑中国股票市场收益率在时间层面上的动量效应。

Model:
$$
r_{t} = \alpha + \beta * r_{t-1} + \epsilon_{t}
$$

where $r_t$ is the raw return of stock market on month $t$.

* H0: $\beta = 0$
* H1: $\beta \ne 0$

In [ ]:
# 使用t-1月的收益率预测t月的收益率
# Newey West 检验 自相关
# 一般在日数据里面，lag12，月lag6，季度lag2

model1_mom = smf.ols('Raw_return ~ lag_Raw_return',
                 data=Month_data['2012-12':'2022-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 6})
print(model1_mom.summary())

所以上述模型的结果是：

$$r_{t} =  +  * r_{t-1} + \epsilon_{t} $$


In [ ]:
Quarter_data['lag_Raw_return'] = Quarter_data['Raw_return'].shift(1)
Quarter_data

In [ ]:
model2_mom = smf.ols('Raw_return ~ lag_Raw_return',
                 data=Quarter_data['2012-12':'2022-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 2})
print(model2_mom.summary())

In [ ]:
daily_data['lag_Raw_return'] = daily_data['Raw_return'].shift(1)
daily_data

In [ ]:
model3_mom = smf.ols('Raw_return ~ lag_Raw_return',
                 data=daily_data['2012-12':'2022-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 2})
print(model3_mom.summary())

### 整合结果

In [ ]:
from statsmodels.iolib.summary2 import summary_col

info_dict = {'No. observations': lambda x: f"{int(x.nobs):d}"}

results_table = summary_col(results=[model1_mom, model2_mom, model3_mom],
                            float_format='%0.3f', #数据显示的格式，默认四位小数
                            stars=True, # 是否有*，True为有
                            model_names=['Month MOM', 'Quarter MOM', 'Daily MOM'],
                            info_dict=info_dict,
                            regressor_order=['Intercept', 'lag_Raw_Return'])

results_table.add_title(
    'Table - OLS Regressions: Forecast Stock Market Return')

print(results_table)

In [1]:
inflation = pd.read_csv('inflation.csv')
inflation['month'] = pd.to_datetime(inflation['month'],format='%Y/%m/%d')
inflation.set_index('month',inplace=True)
inflation.sort_values(by=['month'],axis=0,ascending=True)

NameError: name 'pd' is not defined

工业品出厂价格指数ppi可视化

In [11]:
fig=plt.figure(figsize=(50,5))
plt.plot('ppi',#作图的变量名称
    '^-r', #线的类型
    label='PPI',#标识
    linewidth =1, #线的粗细
    data=Month_data2)
    
plt.title("Producer Price Index")
plt.xlabel('Month')#横坐标标题 x轴
plt.ylabel('PPI')#纵坐标标题 y轴

year_freq=pd.date_range(start='2012',end='2023',freq='Y')
c=plt.xticks(year_freq,year_freq.year,rotation=270,fontsize=10)

NameError: name 'plt' is not defined

# 月度数据的预测

A simple linear regression of an asset return on one or a few lagged predictors of interest is the most popular econometric approach for testing for return predictability. For simplicity, consider a univariate predictive regression of the period- $(t + 1)$ stock market return $r_{t+1}$ on a single predictor variable $x_t$:
$$
r_{t+1}=\alpha+\beta x_{t}+\varepsilon_{t+1}
$$
where $\varepsilon_{t+1}$ is a zero-mean, unpredictable disturbance term. When $x_t$ is the inflation rate, dividend yield, book-to-price ratio, or turnover. Many researchers find that $\beta$ is significantly different from zero; that is, there is in-sample evidence of stock market return predictability.

* H0:$\beta = 0$
* H1:$\beta \ne 0$(我们需要通过理论分析，得出$\beta$的符号)

In [ ]:
Month_data.reset_index(inplace=True)
Month_data

In [ ]:
Month_data.rename(columns={'Day':'month'},inplace=True)
Month_data

In [ ]:
reg_data = pd.merge(Month_data,inflation,left_on='month',right_on='month')
reg_data

In [ ]:
reg_data.set_index('month',inplace=True)
reg_data

In [ ]:
reg_data2 = pd.merge(Month_data,inflation,on = 'month')
reg_data2

In [ ]:
reg_data3 = pd.merge(Month_data,inflation,how='right',on = 'month')
reg_data3

## 描述性统计 Summary


In [ ]:
reg_data['ppi'].describe().round(5)

In [ ]:
reg_data['ppi'].skew()
reg_data['ppi'].kurt()

## OLS 回归结果

In [ ]:
reg_data['lppi'] = reg_data['ppi'].shift(2)
model_ppi = smf.ols('Raw_return ~ lppi',
                 data=reg_data['2012-12':'2022-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 6})
print(model_ppi.summary())

## 预期收益率 Expected Return / Conditional Return

In [9]:
data = reg_data['2012-12':'2022-12'].copy()
data['fitted_return'] =  model_ppi.fittedvalues

fig = plt.figure(figsize=(10, 5))
plt.plot('Raw_return',
         '-r',
         label='ret',
         linewidth=1,
         data=data)
plt.plot('fitted_return',
         '-b',
         label='Fitted Return',
         linewidth=1,
         data=data)
plt.title("China's Stock Market")
plt.xlabel('Month')  # 画图的x轴名称
plt.ylabel('Return')  # 画图的y轴名称

fig = plt.gcf()
fig.savefig('fitted_return.pdf', bbox_inches='tight')  # 更改输出图片格式 jpg

NameError: name 'reg_data' is not defined

## 长期预测 Long Horizon Forecast

$$
r_{t+1} + r_{t+2} + r_{t+3}  =\alpha+\beta x_{t}+\varepsilon_{t+1}
$$

In [ ]:
reg_data['next_ret'] = reg_data['Raw_return'].shift(-1) + 1
reg_data['next_ret2'] = reg_data['Raw_return'].shift(-2) + 1
reg_data['next_ret3'] = reg_data['Raw_return'].shift(-3) + 1
reg_data['future_3month_return'] = reg_data['next_ret'] * reg_data['next_ret2'] * reg_data['next_ret3'] - 1
reg_data

In [ ]:
model_ppi_3month = smf.ols('future_3month_return ~ lppi',
                 data=reg_data['2012-12':'2022-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 6})
print(model_ppi_3month.summary())

# 整理结果

In [ ]:
from statsmodels.iolib.summary2 import summary_col

info_dict = {'No. observations': lambda x: f"{int(x.nobs):d}"}

results_table = summary_col(results=[model_ppi, model_ppi_3month],
                            float_format='%0.3f', #数据显示的格式，默认四位小数
                            stars=True, # 是否有*，True为有
                            model_names=['Month PPI', 'Month future 3 PPI'],
                            info_dict=info_dict,
                            regressor_order=['Intercept', 'lppi'])

results_table.add_title(
    'Table - OLS Regressions: Forecast Stock Market Return')

print(results_table)